In [1]:
import torch
import torch.optim as optim
from datasets import load_dataset
from tools import *
from model import *
from train import *
from plot import *
from eval import *

# https://huggingface.co/datasets/yoshitomo-matsubara/srsd-feynman_easy
dataset = load_dataset('yoshitomo-matsubara/srsd-feynman_easy')

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [2]:
def run_experiment(n_equation, plot=False, save=True, dist=True):
    train_loader, test_loader, features_, labels_, features, labels = prepare_data_loaders(dataset, n_equation, batch_size=batch_size, equation_size=equation_size)
    hist_train_loss, hist_test_loss = train_model(formuler, train_loader, test_loader, optimizer, criterion, scheduler, num_epochs)
    if plot:
        plot_loss_curves(hist_train_loss, hist_test_loss, start=0)
        # Plotting for training data
        x_train, formula_train = formuler(features_.to(device))
        x_train = x_train.detach().cpu()
        print(criterion(x_train.squeeze(1), labels_))
        print(features_.shape)
        plot_grid(features_, x_train, labels_, "Train")

        # Plotting for testing data
        x_test, formula_test = formuler(features.to(device))
        x_test = x_test.detach().cpu()
        print(criterion(x_test.squeeze(1), labels))
        print(features.shape)
        plot_grid(features, x_test, labels, "Test")
        if dist:
            plot_gradient_distributions(formuler, features_, device)

    min_loss, min_for = generate_minimal_loss_formula(formuler, features, features_, labels, device, criterion, model, prob_size=prob_size)
    if plot:
        x = decode(min_for, features.to(device))
        print(criterion(x.squeeze(1).requires_grad_(), labels.to(device)).detach().cpu())
        x = x.detach().cpu()
        plot_grid(features, x, labels, "predicted formula")
    if save:
        torch.save(model, f'saved_models\model-formula-{n_equation}.pth')
    return min_loss, min_for


In [3]:
equation_size = 30
batch_size = 60
num_epochs = 25
prob_size = 4
losses = []
formulas = []
equations = [7]
for n_equation in equations:
    print(n_equation)
    x, y = data_prep(dataset, "test", n_equation, 30)
    input_size = x.shape[1]
    model = [input_size, input_size, input_size, input_size , 3, 2, 1]
    formuler = Formuler(model).to(device)
    optimizer = optim.Adam(formuler.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0001)
    criterion =  torch.nn.MSELoss()
    min_loss, min_for = run_experiment(n_equation, plot=True, save=False, dist=False)
    losses.append(min_loss)
    formulas.append(min_for)

7
EPOCH[0] test loss: 1303.454411, train loss: 3622.854980, weighted_loss: 3622.854980
EPOCH[1] test loss: 1296.263567, train loss: 599.661682, weighted_loss: 599.661682
EPOCH[2] test loss: 1295.865403, train loss: 604.370300, weighted_loss: 604.370300
EPOCH[3] test loss: 1436.325776, train loss: 599.922180, weighted_loss: 599.922180
EPOCH[4] test loss: 23112.074238, train loss: 133385000.000000, weighted_loss: 133385000.000000


In [ ]:
losses, formulas

([1317.4136962890625],
 [[[['g', 0], ['ln', 0], ['cos', 2], ['g', 1]],
   [[0, '/', 2], [0, '/', 1], [2, '+', 3]],
   [['cos', 1], ['cos', 2]],
   [[0, '/', 1]]]])